# Evaluating `pointblank` (Python version)

[Pointblank documentation - Python](https://posit-dev.github.io/pointblank/)

The following types of data tables are supported:

- Polars DataFrame
- Pandas DataFrame
- DuckDB table
- MySQL table
- PostgreSQL table
- SQLite table
- Parquet

Note that `pointblank` uses `Narwhals` to work with `Polars` and `Pandas` DataFrames and also integrates with `Ibis` to enable the use of `DuckDB`, `MySQL`, `PostgreSQL`, `SQLite`, `Parquet`, etc.

So with `pointblank`, you can validate data in a variety of data sources and thus can handle quite large volumes of data efficiently e.g. via tools like `DuckDB`.

## `pointblank` in R

[Pointblank documentation - R](https://rstudio.github.io/pointblank/)

The heritage of `pointblank` is in the R version initially released in 2017. The R version is more focused on the `dplyr` and `dbplyr` packages. The `pointblank` package in R is designed to work with `dplyr` and `dbplyr` data sources, and it can also be used with `dbplyr`-compatible databases.

The Python version appears to be a relatively new project (late 2024).



In [ ]:
#| echo: false

import math
from pathlib import Path
from typing import List, Set

import duckdb
import pandas as pd
import pointblank as pb
from IPython.display import Markdown

## Dataset: Titanic 

The Titianic dataset is a simple (and well-known) example with well-documented data quality issues


In [ ]:
TITANIC_URL = 'https://hbiostat.org/data/repo/titanic3.csv'

In [ ]:
#| echo: false

def get_create_titanic_db(db_path: str = 'titanic.duckdb', url: str = TITANIC_URL, force_create: bool = False) -> duckdb.DuckDBPyConnection:
    """
    Connects to a DuckDB database, creating it if it doesn't exist,
    and loads data from a CSV URL into a table named 'titanic'.  It enhances
    the table by adding a unique 'id' column.

    Args:
        db_path (str, optional): The path to the DuckDB database file.
            Defaults to 'titanic.duckdb'.
        url (str, optional): The URL of the CSV file to load.
            Defaults to TITANIC_URL.
        force_create (bool, optional): If True, the database will be recreated
            even if it already exists. Defaults to False.

    Returns:
        duckdb.DuckDBPyConnection: A connection to the DuckDB database.
    """
    if not Path(db_path).exists() or force_create:
        # Connect to DuckDB and create a persistent database
        con = duckdb.connect(database=db_path, read_only=False)

        # Read the CSV files directly into DuckDB
        con.execute(f"CREATE TABLE IF NOT EXISTS titanic AS SELECT * FROM read_csv('{url}')")
        # Create a new table with a unique identifier
        con.execute("""
            CREATE TABLE titanic_with_id AS 
            SELECT *, ROW_NUMBER() OVER () AS id 
            FROM titanic
        """)
        # Drop the original table and rename the new table
        con.execute("DROP TABLE titanic")
        con.execute("ALTER TABLE titanic_with_id RENAME TO titanic")
    else:
        # Connect to the existing database
        con = duckdb.connect(database=db_path, read_only=False)
    
    return con


Get the data and create (or load) a database version

In [ ]:
#| echo: false

con = get_create_titanic_db()

Quick checks on data

In [ ]:
con.sql("SELECT COUNT(*) AS n_record FROM titanic")

Look at first 5 records

In [ ]:
#| echo: false

con.sql("SELECT * FROM titanic LIMIT 5").df()

In [ ]:
#| echo: false

def create_random_samples(con: duckdb.DuckDBPyConnection, n_sample: int, RANDOM_SEED: int = 42) -> List[pd.DataFrame]:
    """
    Splits the 'titanic' dataset into a specified number of distinct, randomised samples.

    This function aims to simulate a scenario where one sample of the data is used to define 
    data quality or validation rules, and the remaining samples are then evaluated
    against these established rules. This approach allows for testing the robustness
    and generalisability of the rules.

    Args:
        con (duckdb.DuckDBPyConnection): A connection to the DuckDB database containing the 'titanic' table.
        n_sample (int): The number of distinct samples to create from the dataset.

    Returns:
        List[pd.DataFrame]: A list of Pandas DataFrames, where each DataFrame represents a distinct,
        randomized sample from the original 'titanic' dataset. The samples are designed to be 
        used for training and evaluating data quality/validation rules.
    """
    total_rows = con.sql("SELECT COUNT(*) FROM titanic").fetchone()[0]

    # Calculate the number of rows per sample
    rows_per_sample = math.ceil(total_rows / n_sample)

    # Get all rows and shuffle them
    all_rows = con.sql("SELECT * FROM titanic").fetchdf()
    shuffled_rows = all_rows.sample(frac=1, random_state=RANDOM_SEED).reset_index(drop=True)

    samples = []
    sample_size = []
    for i in range(n_sample):
        offset = i * rows_per_sample
        sample = shuffled_rows.iloc[offset:offset + rows_per_sample]
        sample_size.append(len(sample))
        samples.append(sample)

    return samples,sample_size


In [ ]:
N_SAMPLE = 5

In [ ]:
samples, sample_size = create_random_samples(con, N_SAMPLE)

Sizes of each sample

In [ ]:
sample_size

Check the head of the first sample

In [ ]:
samples[0].head()

Cross-check that the samples combine to match the original data

In [ ]:
#| echo: false

# Fetch the original (entire) dataframe from the database
original_df = con.sql("SELECT * FROM titanic").df()

# Concatenate all samples into a single dataframe
concat_samples = pd.concat(samples).sort_values('id').reset_index(drop=True)

# Check if the dataframes are identical
identical = original_df.equals(concat_samples)

print(f"Original dataframe and concatenated samples identical? {'Yes' if identical else 'No'}")


## Pointblank functionality

### `preview`

Shows the head and tail of the data and stats

In [ ]:
pb.preview(original_df)

### Data validation 'thresholds'

In [ ]:
PASSENGER_CLASS: Set[int] = {1, 2, 3}       # Allowed values for passenger class
PORT_EMBARKED: Set[str] = {"C", "Q", "S"}   # Allowed values for port of embarkation
SURVIVED: Set[int] = {0, 1}                 # Allowed values for survival status
SEX: Set[str] = {"male", "female"}          # Allowed values for sex
MIN_AGE: int = 0                            # Minimum allowed age
MAX_AGE: int = 70                           # Maximum allowed age
MIN_FARE: int = 0                           # Minimum allowed fare
MAX_FARE: int = 500                         # Maximum allowed fare

In [ ]:
def validate_titanic_data_sample(data: pd.DataFrame) -> pb.validate.Validate:
    """
    Validates a Pandas DataFrame containing Titanic data using pointblank.

    This function defines a set of validation rules for the Titanic dataset,
    checking for missing values, value ranges, and allowed values in specific columns.

    Args:
        sample_df (pd.DataFrame): The Pandas DataFrame to validate.

    Returns:
        pb.ValidationSummary: A pointblank ValidationSummary object containing the validation results.
    """

    validation = (
        pb.Validate(data=data, label="Titanic Data Validation")
        .col_vals_not_null("survived")
        .col_vals_not_null("pclass")
        .col_vals_not_null("sex")
        .col_vals_not_null("age")
        .col_vals_not_null("ticket")
        .col_vals_not_null("fare")
        .col_vals_not_null("embarked")
        .col_vals_between("age", MIN_AGE, MAX_AGE, na_pass=True)
        .col_vals_between("fare", MIN_FARE, MAX_FARE)
        .col_vals_in_set("pclass", PASSENGER_CLASS)
        .col_vals_in_set("embarked", PORT_EMBARKED)
        .col_vals_in_set("survived", SURVIVED)
        .col_vals_in_set("sex", SEX)
        .interrogate()
    )

    return validation


In [ ]:
validation = validate_titanic_data_sample(original_df)

In [ ]:
validation.get_tabular_report()

In [ ]:
validation.get_sundered_data()

TODO: Use the first sample to "define" the thresholds and then process the remaining samples with thresholds and actions.

https://posit-dev.github.io/pointblank/user-guide/thresholds.html

https://posit-dev.github.io/pointblank/user-guide/actions.html

TODO: Also take a look at working directly with DuckDB data table not via Pandas

### Validate the individual samples

In [ ]:
#| echo: false

for i, sample in enumerate(samples):
    validation = validate_titanic_data_sample(sample)
    display(Markdown(f"**Sample {i+1}**: {len(sample)} rows"))
    display(validation)